In [2]:

from openai import OpenAI
import os

client = OpenAI(
    # 如果没有配置环境变量，请用API Key将下行替换为：api_key="sk-xxx"
    # 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    api_key='sk-dec3caaa6d6d4350963f5ceb97dce549',
    # 以下是北京地域base_url，如果使用新加坡地域的模型，需要将base_url替换为：https://dashscope-intl.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",)

completion = client.chat.completions.create(
    model="qwen3-max-preview",
    messages=[
        {
            "role": "system",
            "content": "這是一個來自儒林外史的片段，請找出其中和蘇州有關的稱呼。只返回稱呼，沒有則返回無。"
        },
        {
            "role": "user",
            "content": "話說應天蘇州府常熟縣有個鄉村，叫做麟紱鎮。鎮上有二百多人家，都是務農為業。只有一位姓虞，在成化年間，讀書進了學，做了三十年的老秀才，只在這鎮上教書。這鎮離城十五里。虞秀才除應考之外，從不到城裏去走一遭，後來直活到八十多歲，就去世了。他兒子不曾進過學，也是教書為業。到了中年，尚無子嗣。夫婦兩個到文昌帝君面前去求，夢見文昌親手遞一紙條與他，上寫著《易經》一句：「君子以果行育德。」當下就有了娠。到十個月滿足，生下這位虞博士來。太翁去謝了文昌，就把這新生的兒子取名育德，字果行。這虞博士三歲上就喪了母親，太翁在人家教書，就帶在館裏，六歲上替他開了蒙。虞博士長到十歲，鎮上有一位姓祁的祁太公包了虞太翁家去教兒子的書，賓主甚是相得。教了四年，虞太翁得病去世了，臨危把虞博士託與祁太公。此時虞博士年方十四歲。祁太公道：「虞小相公比人家一切的孩子不同，如今先生去世，我就請他做先生教兒子的書。」當下寫了自己祁連的名帖，到書房裏來拜，就帶著九歲的兒子來拜虞博士做先生。虞博士自此總在祁家教書。", 
        },
    ],
    extra_body={"enable_thinking": True,'thinking_budget':4000} # 关闭文档检索功能
)

result = completion.choices[0].message.content
print(result)

蘇州府


In [8]:
# ...existing code...
# 简化版批处理
from openai import OpenAI
import os
import os, glob
from concurrent.futures import ThreadPoolExecutor, as_completed

BASE_DIR = r"d:\study\研一上\CHC5904\A2"
INPUT_DIR = os.path.join(BASE_DIR, "30-50")
OUTPUT_DIR = os.path.join(BASE_DIR, "30-50_out_揚州")
os.makedirs(OUTPUT_DIR, exist_ok=True)

SYSTEM_PROMPT = "這是一個來自儒林外史的片段，請找出其中和揚州有關的地名別稱。只返回別稱，沒有則返回無。"
SUMMARY_PATH = os.path.join(OUTPUT_DIR, "_summary.txt")
SKIP_DONE = True
client = OpenAI(
    # 如果没有配置环境变量，请用API Key将下行替换为：api_key="sk-xxx"
    # 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    api_key='sk-dec3caaa6d6d4350963f5ceb97dce549',
    # 以下是北京地域base_url，如果使用新加坡地域的模型，需要将base_url替换为：https://dashscope-intl.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",)
def ask_once(text: str) -> str:
    resp = client.chat.completions.create(
        model="qwen3-max-preview",
        temperature=0.7,
        top_p=0.8,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text},
        ],
    )
    return (resp.choices[0].message.content or "").strip()

def process_file(file_path: str):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f if ln.strip()]  # 跳过空行
    if not lines:
        combined = "無"
    else:
        outs = [ask_once(ln) for ln in lines]
        outs = [o for o in outs if o and o != "無"]  # 只保留称呼
        combined = "無" if not outs else "、".join(outs)  # 简单合并，不去重
    rel = os.path.relpath(file_path, INPUT_DIR)
    out_path = os.path.join(OUTPUT_DIR, rel)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(combined)
    return rel, combined

txt_files = [p for p in glob.glob(os.path.join(INPUT_DIR, "**", "*.txt"), recursive=True)]
to_run, done = [], []
for p in txt_files:
    rel = os.path.relpath(p, INPUT_DIR)
    outp = os.path.join(OUTPUT_DIR, rel)
    if SKIP_DONE and os.path.exists(outp) and os.path.getsize(outp) > 0:
        with open(outp, "r", encoding="utf-8") as f:
            done.append((rel, f.read().strip()))
    else:
        to_run.append(p)

print(f"共 {len(txt_files)} 个文件；跳过 {len(done)} 个；处理 {len(to_run)} 个（8 线程）。")

results = []
with ThreadPoolExecutor(max_workers=8) as ex:
    futures = [ex.submit(process_file, p) for p in to_run]
    for fut in as_completed(futures):
        try:
            results.append(fut.result())
            print("完成：", results[-1][0])
        except Exception as e:
            print("失败：", e)

all_rows = sorted(done + results, key=lambda x: x[0])
with open(SUMMARY_PATH, "w", encoding="utf-8") as f:
    for rel, res in all_rows:
        f.write(f"{rel}\t{res}\n")

print("完成，汇总：", SUMMARY_PATH)
# ...existing code...

共 21 个文件；跳过 0 个；处理 21 个（8 线程）。
完成： Chapter31.txt
完成： Chapter32.txt
完成： Chapter34.txt
完成： Chapter33.txt
完成： Chapter36.txt
完成： Chapter30.txt
完成： Chapter37.txt
失败： Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Input data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-d10f0c9f-7761-4478-a4df-a7f5551c4ef5', 'request_id': 'd10f0c9f-7761-4478-a4df-a7f5551c4ef5'}
完成： Chapter35.txt
完成： Chapter41.txt
完成： Chapter39.txt
完成： Chapter38.txt
完成： Chapter40.txt
完成： Chapter45.txt
完成： Chapter44.txt
完成： Chapter46.txt
完成： Chapter48.txt
完成： Chapter49.txt
完成： Chapter43.txt
完成： Chapter47.txt
完成： Chapter50.txt
完成，汇总： d:\study\研一上\CHC5904\A2\30-50_out_揚州\_summary.txt


In [2]:
import os, glob, csv

BASE_DIR = r"d:\study\研一上\CHC5904\A2"
# 城市 -> 该城市结果所在目录（示例名，按你的实际输出目录修改）
city_dirs = {
    "蘇州": "30-50_out_蘇州",
    "揚州": "30-50_out_揚州",
    "杭州": "30-50_out_杭州",
    "南京": "30-50_out_南京",
    "北京": "30-50_out_北京",
}

out_csv = os.path.join(BASE_DIR, "五城稱呼匯總.csv")
rows = []

for city, dir_name in city_dirs.items():
    dir_path = os.path.join(BASE_DIR, dir_name)
    aliases = set()
    if not os.path.isdir(dir_path):
        rows.append([city, "無"])
        continue

    summary_path = os.path.join(dir_path, "_summary.txt")
    if os.path.exists(summary_path):
        # 优先用汇总文件
        with open(summary_path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.rstrip("\n").split("\t", 1)
                if len(parts) == 2:
                    res = parts[1].strip()
                    if res and res not in ("無", "无"):
                        for a in res.split("、"):
                            a = a.strip()
                            if a and a not in ("無", "无"):
                                aliases.add(a)
    else:
        # 没有汇总文件则读取目录内所有 txt（排除 _summary.txt）
        for p in glob.glob(os.path.join(dir_path, "**", "*.txt"), recursive=True):
            if os.path.basename(p) == "_summary.txt":
                continue
            with open(p, "r", encoding="utf-8") as f:
                res = f.read().strip()
                if res and res not in ("無", "无"):
                    for a in res.split("、"):
                        a = a.strip()
                        if a and a not in ("無", "无"):
                            aliases.add(a)

    combined = "無" if not aliases else "、".join(sorted(aliases))
    rows.append([city, combined])

with open(out_csv, "w", encoding="utf-8-sig", newline="") as f:
    w = csv.writer(f)
    w.writerow(["城市", "称呼"])
    w.writerows(rows)

print("已写入：", out_csv)

已写入： d:\study\研一上\CHC5904\A2\五城稱呼匯總.csv
